In [1]:
import numpy as np
import sqlite3
import pandas as pd
import requests
from astropy.io import fits
import astropy
from astropy.table import Table

import string #call eng alphabet

# 1. Data preparing

## 1.1. Download Data Bases

### 1.1.1. Full Data Base

#### Download the SQLite file from the GRBweb webpage

In [ ]:
r = requests.get("https://icecube.wisc.edu/~grbweb_public/GRBweb2.sqlite")
f = open('GRBweb2.sqlite', 'wb').write(r.content)

#### Load the database with the sqlite3 module

In [ ]:
db = sqlite3.connect('GRBweb2.sqlite')

#### Print the names of all the tables
From the list of table names, select the one you wish to load.From the list of table names, select the one you wish to load.

In [ ]:
table_names = pd.read_sql_query("SELECT * from sqlite_sequence", db)
print("Table names:\n", table_names, "\n\n")

### 1.1.2. Swift table

In [ ]:
swift_table = pd.read_sql_query("SELECT * from Swift", db)
swift_table = swift_table.sort_values("GRB_name").reset_index().drop("index",axis=1)
swift_table

In [ ]:
swift_table.info()

### 1.1.3. Fermi table

In [ ]:
fermi_table = pd.read_sql_query("SELECT * from Fermi_GBM", db)
fermi_table = fermi_table.sort_values('GRB_name_Fermi').reset_index().drop("index",axis=1)
fermi_table

In [ ]:
fermi_table.info()

### 1.1.3. Summary table

In [ ]:
summary_table = pd.read_sql_query("SELECT * from Summary", db)

In [ ]:
summary_table.to_csv("summary_table_grb.csv")

In [ ]:
summary_table.info()

## 1.2. Data reduction (using online tables, save them and one local unite table)

* Preparing to cross matching -- create new names (Fermi style -> Swift style)
* Cross matching (Fermi-Swift)
* Creating new larger table 

### 1.2.1. Preparing to cross matching

In [ ]:
alphabet = list(string.ascii_uppercase)
dictionary = {}
for i in range(0,len(fermi_table)):
    #print(i)
    grb_one_date = []
    for j in range(0,len(fermi_table)):
        if fermi_table['GRB_name_Fermi'].iloc[i][:9] == fermi_table['GRB_name_Fermi'].iloc[j][:9]:
            grb_one_date.append(fermi_table['GRB_name_Fermi'].iloc[j])
    
    grb_one_date.sort()
    #print(grb_one_date)

    for n in range(0,len(grb_one_date)):
        dictionary[grb_one_date[n]] = grb_one_date[n][:9] + alphabet[n]
    #print(dictionary)
    #print("_____________________________")



### 1.2.2. Cross match

In [ ]:
fermi_table['GRB_name_Swift'] = ""*len(fermi_table)
for i in range(0,len(fermi_table)):
    fermi_table['GRB_name_Swift'].iloc[i] = dictionary[fermi_table['GRB_name_Fermi'].iloc[i]]

In [ ]:
fermi_table

### 1.2.3. Creating new table

In [ ]:
sf_table = swift_table #make copy of Swift table to add here 
# additional tables from Fermi catalogue (cross matching)

sf_table

In [ ]:
sf_table["GRB_name_Swift"] = sf_table["GRB_name"]

In [ ]:
sf_table = sf_table.merge(fermi_table, how="left", on="GRB_name_Swift", suffixes=('_swift', '_fermi'))
sf_table

In [ ]:
fermi_table.to_csv("grb_fermi_catalogue")
swift_table.to_csv("grb_swift_catalogue")
sf_table.to_csv("grb_swift_fermi_catalogue_for_ml")

## 1.3. Data reduction (using local summary table)

In [2]:
data = pd.read_csv("summary_table_grb.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8028 entries, 0 to 8027
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            8028 non-null   int64  
 1   id                    8028 non-null   int64  
 2   GRB_name              8028 non-null   object 
 3   GRB_name_Fermi        3435 non-null   object 
 4   T0                    8028 non-null   float64
 5   T0_source             8028 non-null   object 
 6   ra                    8028 non-null   float64
 7   ra_source             8028 non-null   object 
 8   decl                  8028 non-null   float64
 9   decl_source           8025 non-null   object 
 10  pos_error             7786 non-null   float64
 11  pos_error_source      7786 non-null   object 
 12  T90                   7013 non-null   float64
 13  T90_source            7013 non-null   object 
 14  T90_error             6005 non-null   float64
 15  T90_error_source     

In [4]:
#we have to edit name of GRBs because somewhere may be spaces 
#and if somewhere the is lower case we have to fix it on upper case
for ind,name in enumerate(data["GRB_name_Fermi"]):
    data["GRB_name_Fermi"].iloc[ind] = str(name).upper().strip()

/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_88878/3069133779.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["GRB_name_Fermi"].iloc[ind] = str(name).upper().strip()


In [13]:
#to use data in Fermi catalogue we have to drop rows with no Ferni name of GRBs

data_nonull = data[data["GRB_name_Fermi"].notnull()]

#because then we will set names of GRBs as indexes and there should not be NULL value
#there will not be an error, but it is better to protect it from possible errors

data_redshift=data_nonull[["GRB_name_Fermi","redshift"]].set_index("GRB_name_Fermi")

## 1.4. Data reduction (Fermi table)

In [14]:
fermi_astrotable = Table(fits.open("fermi_full_2022.fits")[1].data)
fermi_full = fermi_astrotable.to_pandas()

In [24]:
fermi_astrotable.columns

<TableColumns names=('NAME','RA','DEC','TRIGGER_TIME','T90','T90_ERROR','T90_START','FLUENCE','FLUENCE_ERROR','FLUX_1024','FLUX_1024_ERROR','FLUX_1024_TIME','FLUX_64','FLUX_64_ERROR','FLNC_BAND_AMPL','FLNC_BAND_AMPL_POS_ERR','FLNC_BAND_AMPL_NEG_ERR','FLNC_BAND_EPEAK','FLNC_BAND_EPEAK_POS_ERR','FLNC_BAND_EPEAK_NEG_ERR','FLNC_BAND_ALPHA','FLNC_BAND_ALPHA_POS_ERR','FLNC_BAND_ALPHA_NEG_ERR','FLNC_BAND_BETA','FLNC_BAND_BETA_POS_ERR','FLNC_BAND_BETA_NEG_ERR','FLNC_SPECTRUM_START','FLNC_SPECTRUM_STOP','PFLX_BEST_FITTING_MODEL','PFLX_BEST_MODEL_REDCHISQ','FLNC_BEST_FITTING_MODEL','FLNC_BEST_MODEL_REDCHISQ','TRIGGER_NAME','LII','BII','ERROR_RADIUS','DURATION_ENERGY_LOW','DURATION_ENERGY_HIGH','BACK_INTERVAL_LOW_START','BACK_INTERVAL_LOW_STOP','BACK_INTERVAL_HIGH_START','BACK_INTERVAL_HIGH_STOP','T50','T50_ERROR','T50_START','BCAT_DETECTOR_MASK','FLU_LOW','FLU_HIGH','FLUENCE_BATSE','FLUENCE_BATSE_ERROR','FLUX_64_TIME','FLUX_256','FLUX_256_ERROR','FLUX_256_TIME','FLUX_BATSE_1024','FLUX_BATSE_1024

In [15]:
fermi_full = fermi_full.rename(columns={"NAME":"GRB_name_Fermi"})#we rename the column because
#it another table (data_redshift -- summary table from icecube) it is named in this way

#we have to edit name of GRBs because somewhere may be spaces 
#and if somewhere the is lower case we have to fix it on upper case
for ind,name in enumerate(fermi_full["GRB_name_Fermi"]):
    fermi_full["GRB_name_Fermi"].iloc[ind] = str(name).upper().strip()

/var/folders/kv/prhthx396zd4k1d7f55bdztc0000gn/T/ipykernel_88878/3167320858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fermi_full["GRB_name_Fermi"].iloc[ind] = str(name).upper().strip()


In [19]:
#we join two tables with known redshifts and another data on events

fermi_rs = fermi_full.join(data_redshift,how="left",on="GRB_name_Fermi")

#and remove GRBs without known redshift

fermi_rs = fermi_rs[fermi_rs["redshift"].notnull()]

In [27]:
#now we schould remove unnecessary for ml columns 

fermi_rs = fermi_rs[['GRB_name_Fermi','RA','DEC',
                     'T90',
                     'FLUENCE',
                     'FLUX_1024',
                     'FLUX_1024_TIME','FLUX_64',
                     
                     'FLNC_BAND_AMPL',
                     'FLNC_BAND_EPEAK',
                     'FLNC_BAND_ALPHA',
                     'FLNC_BAND_BETA',
                     'FLNC_SPECTRUM_START',
                     'FLNC_SPECTRUM_STOP',
                     
                     'PFLX_BEST_FITTING_MODEL',
                     'PFLX_BEST_MODEL_REDCHISQ',
                     
                     'FLNC_BEST_FITTING_MODEL',
                     'FLNC_BEST_MODEL_REDCHISQ',
                     
                     'TRIGGER_NAME',
                     'LII','BII',
                     'DURATION_ENERGY_LOW',
                     'DURATION_ENERGY_HIGH',
                     'BACK_INTERVAL_LOW_START',
                     'BACK_INTERVAL_LOW_STOP',
                     'BACK_INTERVAL_HIGH_START',
                     'BACK_INTERVAL_HIGH_STOP',
                     'T50',
                     'BCAT_DETECTOR_MASK',
                     'FLU_LOW',
                     'FLU_HIGH',
                     
                     'FLUENCE_BATSE',
                    
                     'FLUX_64_TIME',
                     'FLUX_256',
                     'FLUX_256_TIME',
                     'FLUX_BATSE_1024',
                     'FLUX_BATSE_1024_TIME',
                     'FLUX_BATSE_64',
                     'FLUX_BATSE_64_TIME',
                     'FLUX_BATSE_256',
                     'FLUX_BATSE_256_TIME',
                     
                     'ACTUAL_64MS_INTERVAL',
                     'ACTUAL_256MS_INTERVAL',
                     'ACTUAL_1024MS_INTERVAL',
                     
                     'SCAT_DETECTOR_MASK',
                     'PFLX_SPECTRUM_START',
                     'PFLX_SPECTRUM_STOP',
                     'PFLX_PLAW_AMPL',
                     'PFLX_PLAW_PIVOT',
                     'PFLX_PLAW_INDEX',
                     'PFLX_PLAW_PHTFLUX',
                     'PFLX_PLAW_PHTFLNC',
                     'PFLX_PLAW_ERGFLUX',
                     'PFLX_PLAW_ERGFLNC',
                     'PFLX_PLAW_PHTFLUXB',
                     'PFLX_PLAW_PHTFLNCB',
                     'PFLX_PLAW_ERGFLNCB',
                     'PFLX_PLAW_REDCHISQ',
                     'PFLX_PLAW_REDFITSTAT',
                     'PFLX_PLAW_DOF',
                     'PFLX_PLAW_STATISTIC',
                     
                     'PFLX_COMP_AMPL',
                     'PFLX_COMP_EPEAK',
                     'PFLX_COMP_INDEX',
                     'PFLX_COMP_PIVOT',
                     'PFLX_COMP_PHTFLUX',
                     'PFLX_COMP_PHTFLNC',
                     'PFLX_COMP_ERGFLUX',
                     'PFLX_COMP_ERGFLNC',
                     'PFLX_COMP_PHTFLUXB',
                     'PFLX_COMP_PHTFLNCB',
                     'PFLX_COMP_ERGFLNCB',
                     'PFLX_COMP_REDCHISQ',
                     'PFLX_COMP_REDFITSTAT',
                     'PFLX_COMP_DOF',
                     'PFLX_COMP_STATISTIC',
                     
                     'PFLX_BAND_AMPL',
                     'PFLX_BAND_EPEAK',
                     'PFLX_BAND_ALPHA',
                     'PFLX_BAND_BETA',
                     'PFLX_BAND_PHTFLUX',
                     'PFLX_BAND_PHTFLNC',
                     'PFLX_BAND_ERGFLUX',
                     'PFLX_BAND_ERGFLNC',
                     'PFLX_BAND_PHTFLUXB',
                     'PFLX_BAND_PHTFLNCB',
                     'PFLX_BAND_ERGFLNCB',
                     'PFLX_BAND_REDCHISQ',
                     'PFLX_BAND_REDFITSTAT',
                     'PFLX_BAND_DOF',
                     'PFLX_BAND_STATISTIC',
                     
                     'PFLX_SBPL_AMPL',
                     'PFLX_SBPL_PIVOT',
                     'PFLX_SBPL_INDX1',
                     'PFLX_SBPL_BRKEN',
                     'PFLX_SBPL_BRKSC',
                     'PFLX_SBPL_INDX2',
                     'PFLX_SBPL_PHTFLUX',
                     'PFLX_SBPL_PHTFLNC',
                     'PFLX_SBPL_ERGFLUX',
                     'PFLX_SBPL_ERGFLNC',
                     'PFLX_SBPL_PHTFLUXB',
                     'PFLX_SBPL_PHTFLNCB',
                     'PFLX_SBPL_ERGFLNCB',
                     'PFLX_SBPL_REDCHISQ',
                     'PFLX_SBPL_REDFITSTAT',
                     'PFLX_SBPL_DOF',
                     'PFLX_SBPL_STATISTIC',
                     
                     'FLNC_PLAW_AMPL',
                     'FLNC_PLAW_PIVOT',
                     'FLNC_PLAW_INDEX',
                     'FLNC_PLAW_PHTFLUX',
                     'FLNC_PLAW_PHTFLNC',
                     'FLNC_PLAW_ERGFLUX',
                     'FLNC_PLAW_ERGFLNC',
                     'FLNC_PLAW_PHTFLUXB',
                     'FLNC_PLAW_PHTFLNCB',
                     'FLNC_PLAW_ERGFLNCB',
                     'FLNC_PLAW_REDCHISQ',
                     'FLNC_PLAW_REDFITSTAT',
                     'FLNC_PLAW_DOF',
                     'FLNC_PLAW_STATISTIC',
                     
                     'FLNC_COMP_AMPL',
                     'FLNC_COMP_EPEAK','FLNC_COMP_INDEX',
                     'FLNC_COMP_PIVOT',
                     'FLNC_COMP_PHTFLUX',
                     'FLNC_COMP_PHTFLNC',
                     'FLNC_COMP_ERGFLUX',
                     'FLNC_COMP_ERGFLNC',
                     'FLNC_COMP_PHTFLUXB',
                     'FLNC_COMP_PHTFLNCB',
                     'FLNC_COMP_ERGFLNCB',
                     'FLNC_COMP_REDCHISQ',
                     'FLNC_COMP_REDFITSTAT',
                     'FLNC_COMP_DOF',
                     'FLNC_COMP_STATISTIC',
                     
                     'FLNC_BAND_PHTFLUX',
                     'FLNC_BAND_PHTFLNC',
                     'FLNC_BAND_ERGFLUX',
                     'FLNC_BAND_ERGFLNC',
                     'FLNC_BAND_PHTFLUXB',
                     'FLNC_BAND_PHTFLNCB',
                     'FLNC_BAND_ERGFLNCB',
                     'FLNC_BAND_REDCHISQ',
                     'FLNC_BAND_REDFITSTAT',
                     'FLNC_BAND_DOF',
                     'FLNC_BAND_STATISTIC',
                     
                     'FLNC_SBPL_AMPL',
                     'FLNC_SBPL_AMPL_POS_ERR',
                     'FLNC_SBPL_PIVOT',
                     'FLNC_SBPL_INDX1',
                     'FLNC_SBPL_BRKEN',
                     'FLNC_SBPL_BRKSC',
                     'FLNC_SBPL_INDX2',
                     'FLNC_SBPL_PHTFLUX', 
                     'FLNC_SBPL_PHTFLNC',
                     'FLNC_SBPL_ERGFLUX', 
                     'FLNC_SBPL_ERGFLNC', 
                     'FLNC_SBPL_PHTFLUXB',
                     'FLNC_SBPL_PHTFLNCB',
                     'FLNC_SBPL_ERGFLNCB',
                     'FLNC_SBPL_REDCHISQ',
                     'FLNC_SBPL_REDFITSTAT',
                     'FLNC_SBPL_DOF',
                     'FLNC_SBPL_STATISTIC',
                     
                     'BCATALOG',
                     'SCATALOG']]

# 2. Machine learning

## 2.1. Linear Regression

### 2.1.1. Linear Regression

### 2.1.2. Lasso

### 2.1.3. Ridge

### 2.1.4. Elastic Net

## 2.2. Support Vector Machine (SVM)

## 2.3. Decision Tree

## 2.4. Random Forest

## 2.5. Boosted Trees